# 🧠 Neural SDE: Learning Market Dynamics from Real Data

## 목표 / Goal
Heston Model(수학 공식)을 버리고, **S&P 500 실제 데이터**로부터 SDE의 Drift/Diffusion 함수를 학습합니다.

Instead of using the Heston Model (mathematical formula), we **learn the Drift/Diffusion functions** directly from real S&P 500 data.

## 왜 Neural SDE인가? / Why Neural SDE?
- **GAN**: 그럴듯한 데이터를 생성하지만, 확률 밀도(PDF)를 모름 → Girsanov 불가
- **Neural SDE**: SDE 구조를 유지하면서 함수를 학습 → **Girsanov Theorem 호환** → NPI 적용 가능

In [ ]:
# =============================================================================
# 환경 설정 / Environment Setup
# =============================================================================
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'

import sys
sys.path.append('..')

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# torchsde 설치 여부 확인 / Check if torchsde is installed
try:
    import torchsde
    print("torchsde is available ✅")
except ImportError:
    print("⚠️ torchsde is not installed. Run: pip install torchsde")
    print("   We will use a custom implementation instead.")

## 1. 데이터 준비 / Data Preparation

S&P 500 (SPY ETF) 일별 수익률을 로드합니다.

Load S&P 500 (SPY ETF) daily returns.

In [ ]:
# =============================================================================
# 데이터 로드 / Load Data
# 이미 저장된 데이터가 있으면 로드, 없으면 yfinance로 다운로드
# Load saved data if exists, otherwise download via yfinance
# =============================================================================

data_path = '../data/processed/spy_returns.csv'

if os.path.exists(data_path):
    print(f"Loading data from {data_path}...")
    df = pd.read_csv(data_path, index_col=0, parse_dates=True)
else:
    print("Downloading SPY data from yfinance...")
    try:
        import yfinance as yf
        spy = yf.download('SPY', start='2010-01-01', end='2024-12-31')
        df = pd.DataFrame()
        df['Close'] = spy['Close']
        df['Returns'] = df['Close'].pct_change()
        df = df.dropna()
        
        # 저장 / Save
        os.makedirs('../data/processed', exist_ok=True)
        df.to_csv(data_path)
        print(f"Data saved to {data_path}")
    except ImportError:
        print("⚠️ yfinance not installed. Using synthetic data.")
        # 합성 데이터 생성 / Generate synthetic data
        np.random.seed(42)
        n_days = 3500
        returns = np.random.normal(0.0003, 0.012, n_days)  # 평균 수익률 0.03%, 변동성 1.2%
        # 가끔 큰 하락 추가 (Fat Tail 시뮬레이션) / Add occasional large drops
        crash_days = np.random.choice(n_days, size=50, replace=False)
        returns[crash_days] = np.random.normal(-0.03, 0.02, 50)
        df = pd.DataFrame({'Returns': returns})

print(f"Data shape: {df.shape}")
print(df.describe())

In [ ]:
# =============================================================================
# 수익률 분포 시각화 / Visualize Return Distribution
# =============================================================================

returns = df['Returns'].values

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 히스토그램 / Histogram
axes[0].hist(returns, bins=100, color='steelblue', alpha=0.7, density=True)
axes[0].set_xlabel('Daily Return')
axes[0].set_ylabel('Density')
axes[0].set_title('SPY Daily Return Distribution')
axes[0].axvline(x=0, color='red', linestyle='--')

# 누적 수익률 곡선 / Cumulative Return
cumulative = (1 + df['Returns']).cumprod()
axes[1].plot(cumulative.values, color='green')
axes[1].set_xlabel('Days')
axes[1].set_ylabel('Cumulative Return')
axes[1].set_title('SPY Cumulative Return (2010-2024)')

plt.tight_layout()
plt.show()

# 통계 / Statistics
print(f"\n📊 Statistics:")
print(f"  Mean daily return: {returns.mean()*100:.4f}%")
print(f"  Std (volatility):  {returns.std()*100:.4f}%")
print(f"  Annualized vol:    {returns.std()*np.sqrt(252)*100:.2f}%")
print(f"  Skewness:          {pd.Series(returns).skew():.4f}")
print(f"  Kurtosis:          {pd.Series(returns).kurtosis():.4f}")

## 2. Neural SDE 모델 정의 / Define Neural SDE Model

**SDE 구조 / SDE Structure:**
$$dS_t = \mu_{\theta}(S_t, t) \cdot S_t \cdot dt + \sigma_{\phi}(S_t, t) \cdot S_t \cdot dW_t$$

- $\mu_{\theta}$: Neural Network (Drift 함수 학습)
- $\sigma_{\phi}$: Neural Network (Diffusion 함수 학습, Softplus로 양수 보장)

In [ ]:
# =============================================================================
# Neural SDE 클래스 정의 (Custom Implementation)
# Define Neural SDE Class (Custom Implementation without torchsde)
# =============================================================================

class NeuralDrift(nn.Module):
    """
    Drift 함수를 학습하는 신경망.
    Neural network that learns the drift function.
    """
    def __init__(self, hidden_dim=64):
        super().__init__()
        # 입력: (log_S, t) → 출력: mu
        # Input: (log_S, t) → Output: mu
        self.net = nn.Sequential(
            nn.Linear(2, hidden_dim),
            nn.SiLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.SiLU(),
            nn.Linear(hidden_dim, 1)
        )
    
    def forward(self, log_S, t):
        """
        Args:
            log_S: 로그 주가 (batch,) / Log price
            t: 시간 (batch,) / Time
        Returns:
            mu: Drift 값 (batch,) / Drift value
        """
        x = torch.stack([log_S, t], dim=-1)
        return self.net(x).squeeze(-1)


class NeuralDiffusion(nn.Module):
    """
    Diffusion 함수를 학습하는 신경망.
    Neural network that learns the diffusion function.
    """
    def __init__(self, hidden_dim=64):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(2, hidden_dim),
            nn.SiLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.SiLU(),
            nn.Linear(hidden_dim, 1)
        )
    
    def forward(self, log_S, t):
        """
        Args:
            log_S: 로그 주가 (batch,) / Log price
            t: 시간 (batch,) / Time
        Returns:
            sigma: Diffusion 값 (batch,), 항상 양수 / Diffusion value, always positive
        """
        x = torch.stack([log_S, t], dim=-1)
        raw = self.net(x).squeeze(-1)
        # Softplus로 양수 보장 / Ensure positive via Softplus
        return F.softplus(raw) + 0.01  # 최소값 0.01 / Minimum 0.01


class NeuralSDEModel(nn.Module):
    """
    Neural SDE 전체 모델.
    Complete Neural SDE model.
    """
    def __init__(self, hidden_dim=64, device='cuda'):
        super().__init__()
        self.drift_net = NeuralDrift(hidden_dim).to(device)
        self.diff_net = NeuralDiffusion(hidden_dim).to(device)
        self.device = device
    
    def simulate(self, S0, T, dt, num_paths):
        """
        Neural SDE로 경로 시뮬레이션.
        Simulate paths using Neural SDE.
        
        Args:
            S0: 초기 주가 / Initial price
            T: 만기 / Maturity
            dt: 시간 간격 / Time step
            num_paths: 경로 수 / Number of paths
        
        Returns:
            S: 주가 경로 (num_paths, n_steps+1) / Price paths
        """
        n_steps = int(T / dt)
        
        # 초기화 / Initialize
        log_S = torch.log(torch.full((num_paths,), S0, device=self.device))
        
        # 경로 저장 / Store paths
        log_S_paths = [log_S.clone()]
        
        # Euler-Maruyama 적분 / Euler-Maruyama integration
        for step in range(n_steps):
            t = step * dt
            t_tensor = torch.full((num_paths,), t, device=self.device)
            
            # Drift & Diffusion 계산 / Compute drift & diffusion
            mu = self.drift_net(log_S, t_tensor)
            sigma = self.diff_net(log_S, t_tensor)
            
            # Brownian motion
            dW = torch.randn(num_paths, device=self.device) * np.sqrt(dt)
            
            # SDE 업데이트 (로그 공간) / SDE update (log space)
            # d(log S) = (mu - 0.5*sigma^2) dt + sigma dW (Itô's Lemma)
            log_S = log_S + (mu - 0.5 * sigma**2) * dt + sigma * dW
            log_S_paths.append(log_S.clone())
        
        # 스택 및 지수 변환 / Stack and exponentiate
        log_S_paths = torch.stack(log_S_paths, dim=1)
        S_paths = torch.exp(log_S_paths)
        
        return S_paths

print("Neural SDE Model defined ✅")

## 3. Neural SDE 학습 / Training Neural SDE

**학습 방법 / Training Method:**
- **Maximum Likelihood Estimation (MLE)**: 실제 수익률이 나올 확률을 최대화
- **Loss**: 음의 로그 우도 (Negative Log-Likelihood, NLL)

In [ ]:
# =============================================================================
# 학습 데이터 준비 / Prepare Training Data
# =============================================================================

# 수익률을 텐서로 변환 / Convert returns to tensor
returns_tensor = torch.tensor(returns, dtype=torch.float32, device=device)

# 가상의 일간 시간 축 (0~1 사이로 정규화) / Normalized daily time axis
n_days = len(returns)
time_tensor = torch.linspace(0, 1, n_days, device=device)

# 로그 가격 복원 (누적 합) / Reconstruct log prices (cumulative sum)
log_prices = torch.zeros(n_days + 1, device=device)
log_prices[0] = np.log(100)  # 초기 로그 가격 / Initial log price
for i in range(n_days):
    log_prices[i+1] = log_prices[i] + returns_tensor[i]

print(f"Training data prepared: {n_days} days of returns")

In [ ]:
# =============================================================================
# 학습 루프 / Training Loop
# =============================================================================

# 모델 초기화 / Initialize model
neural_sde = NeuralSDEModel(hidden_dim=64, device=device)
optimizer = torch.optim.Adam(neural_sde.parameters(), lr=1e-3)

# 학습 설정 / Training settings
num_epochs = 500
batch_size = 512
dt = 1.0 / 252  # 일간 시간 간격 / Daily time step

loss_history = []

print("Training Neural SDE on real market data...")
for epoch in range(num_epochs):
    optimizer.zero_grad()
    
    # 랜덤 배치 샘플링 / Random batch sampling
    idx = torch.randint(0, n_days - 1, (batch_size,), device=device)
    
    # 현재 로그 가격과 수익률 / Current log prices and returns
    log_S_t = log_prices[idx]
    r_observed = returns_tensor[idx]  # 관측된 수익률 / Observed returns
    t_batch = time_tensor[idx]
    
    # Neural SDE 예측 / Neural SDE prediction
    mu = neural_sde.drift_net(log_S_t, t_batch)
    sigma = neural_sde.diff_net(log_S_t, t_batch)
    
    # 예측 수익률 분포 / Predicted return distribution
    # r ~ N(mu*dt, sigma^2*dt) (근사) / r ~ N(mu*dt, sigma^2*dt) (approximation)
    predicted_mean = mu * dt
    predicted_std = sigma * np.sqrt(dt)
    
    # NLL Loss (Gaussian) / NLL Loss (Gaussian assumption)
    nll = 0.5 * ((r_observed - predicted_mean) / predicted_std) ** 2 + torch.log(predicted_std)
    loss = nll.mean()
    
    loss.backward()
    torch.nn.utils.clip_grad_norm_(neural_sde.parameters(), 1.0)
    optimizer.step()
    
    loss_history.append(loss.item())
    
    if epoch % 100 == 0:
        avg_mu = mu.mean().item()
        avg_sigma = sigma.mean().item()
        print(f"Epoch {epoch:4d}: Loss={loss.item():.4f}, μ={avg_mu:.4f}, σ={avg_sigma:.4f}")

print("Training Complete!")

In [ ]:
# =============================================================================
# 학습 곡선 시각화 / Visualize Training Curve
# =============================================================================

plt.figure(figsize=(10, 4))
plt.plot(loss_history, color='blue')
plt.xlabel('Epoch')
plt.ylabel('NLL Loss')
plt.title('Neural SDE Training Loss')
plt.grid(True, alpha=0.3)
plt.show()

## 4. Neural SDE 검증 / Validate Neural SDE

학습된 Neural SDE로 경로를 생성하고, 실제 SPY 데이터와 비교합니다.

Generate paths with the trained Neural SDE and compare with real SPY data.

In [ ]:
# =============================================================================
# 경로 생성 및 비교 / Generate Paths and Compare
# =============================================================================

# 시뮬레이션 설정 / Simulation settings
S0 = 100.0
T = 1.0  # 1년 / 1 year
dt = 1.0 / 252
num_paths = 10000

# Neural SDE로 경로 생성 / Generate paths with Neural SDE
neural_sde.eval()
with torch.no_grad():
    S_neural = neural_sde.simulate(S0, T, dt, num_paths)

S_neural_np = S_neural.cpu().numpy()

# 수익률 추출 / Extract returns
neural_returns = np.diff(np.log(S_neural_np), axis=1).flatten()

# 비교 시각화 / Comparison visualization
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 히스토그램 비교 / Histogram comparison
axes[0].hist(returns, bins=100, alpha=0.5, label='Real SPY', density=True, color='blue')
axes[0].hist(neural_returns, bins=100, alpha=0.5, label='Neural SDE', density=True, color='red')
axes[0].set_xlabel('Daily Return')
axes[0].set_ylabel('Density')
axes[0].set_title('Return Distribution: Real vs Neural SDE')
axes[0].legend()

# 경로 샘플 시각화 / Sample path visualization
for i in range(20):
    axes[1].plot(S_neural_np[i], alpha=0.3, linewidth=0.5)
axes[1].set_xlabel('Days')
axes[1].set_ylabel('Price')
axes[1].set_title('Neural SDE Sample Paths (1 Year)')

plt.tight_layout()
plt.show()

# 통계 비교 / Statistical comparison
print(f"\n📊 Statistical Comparison:")
print(f"  {'Metric':<20} {'Real SPY':>12} {'Neural SDE':>12}")
print(f"  {'-'*44}")
print(f"  {'Mean Return':<20} {returns.mean()*100:>11.4f}% {neural_returns.mean()*100:>11.4f}%")
print(f"  {'Std (Volatility)':<20} {returns.std()*100:>11.4f}% {neural_returns.std()*100:>11.4f}%")
print(f"  {'Skewness':<20} {pd.Series(returns).skew():>12.4f} {pd.Series(neural_returns).skew():>12.4f}")
print(f"  {'Kurtosis':<20} {pd.Series(returns).kurtosis():>12.4f} {pd.Series(neural_returns).kurtosis():>12.4f}")

## 5. Neural SDE + NPI: Crash Generation

학습된 Neural SDE를 기반으로 **DriftNet을 활용한 폭락 생성**을 수행합니다.

Use the trained Neural SDE with **DriftNet for crash generation**.

⚠️ **Note**: 이 부분은 기존 Heston 기반 시뮬레이터와 Neural SDE를 통합하는 고급 기능입니다. 추후 확장 예정입니다.

In [ ]:
# =============================================================================
# Neural SDE 기반 폭락 분석 (간단 버전)
# Crash Analysis with Neural SDE (Simple Version)
# =============================================================================

# 20% 폭락 임계점 / 20% crash threshold
CRASH_THRESHOLD = 0.80
K_crash = CRASH_THRESHOLD * S0

# Terminal 가격 추출 / Extract terminal prices
S_T_neural = S_neural_np[:, -1]

# 폭락 확률 계산 / Crash probability
crash_mask = S_T_neural < K_crash
crash_prob = crash_mask.mean()

print(f"\n🦢 Neural SDE Crash Analysis (1 Year Horizon)")
print(f"="*50)
print(f"Crash Threshold: ${K_crash:.2f} (20% drop from ${S0:.2f})")
print(f"Crash Probability: {crash_prob*100:.2f}%")
print(f"Crash Events: {crash_mask.sum()} / {num_paths}")

# Terminal 분포 시각화 / Visualize terminal distribution
plt.figure(figsize=(10, 5))
plt.hist(S_T_neural, bins=100, color='steelblue', alpha=0.7)
plt.axvline(x=K_crash, color='red', linestyle='--', linewidth=2, label=f'Crash Threshold ({K_crash})')
plt.axvline(x=S0, color='gray', linestyle=':', linewidth=2, label=f'Initial Price ({S0})')
plt.xlabel('Terminal Price')
plt.ylabel('Frequency')
plt.title('Neural SDE: Terminal Price Distribution (1 Year)')
plt.legend()
plt.show()

print(f"\n💡 Insight:")
print(f"   Neural SDE가 실제 시장 데이터로부터 학습한 동역학을 기반으로")
print(f"   폭락 확률을 추정했습니다. 이는 Heston Model의 수학적 가정에")
print(f"   의존하지 않는 Data-Driven 접근법입니다.")

## 📝 결론 / Conclusion

**Neural SDE**를 통해 실제 S&P 500 데이터로부터 시장 동역학(Drift, Diffusion)을 학습했습니다.

We learned market dynamics (Drift, Diffusion) from real S&P 500 data using **Neural SDE**.

**주요 성과 / Key Achievements:**
1. Heston Model 없이 데이터 기반 SDE 함수 학습
2. 실제 수익률 분포와 유사한 시뮬레이션 결과
3. Fat Tail 및 비대칭성(Skewness)을 더 잘 포착

**다음 단계 / Next Steps:**
- Neural SDE + NPI Control (DriftNet) 통합
- Girsanov Weight 계산 확장
- 실제 위기 시나리오 생성 및 비교